In [1]:
# pip install bitsandbytes accelerate flash-attn

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [8]:
import torch
import bz2
import pickle
import importlib
from tqdm import tqdm


import time
from IPython.display import JSON
import sys
sys.settrace(None)
import pdb

import copy

import psycopg
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, StoppingCriteria, StoppingCriteriaList, DynamicCache, OffloadedCache #, CodeGenTokenizer
from transformers.generation.logits_process import LogitsProcessorList

In [9]:
assert torch.cuda.is_available()

In [10]:
### Remind to remove '.py'
# index_module = importlib.import_module('phi4_index')
#index_module = importlib.import_module('http_index')
index_module = importlib.import_module('qwen25_index')
#index_module = importlib.import_module('gemma3_index')
#index_module = importlib.import_module('llama_index')
index_config = getattr(index_module, 'index_config')

# model_module = importlib.import_module('phi4_model')
model_module = importlib.import_module('qwen25_1B_model')
#model_module = importlib.import_module('gemma3_4B_model')
#model_module = importlib.import_module('llama_1B_model')
model_config = getattr(model_module, 'model_config')

In [11]:
streamer = TextStreamer(model_config.tokenizer)

In [12]:
from ctrie import ConstrainedLogitsProcessor, ConstrainedStateList, \
                    ConstrainedState, DictIndex

In [13]:
question = '''Which city is the capital of the country where the Eiffel Tower is?'''
question = '''Who was the quarterback of the team that won Super Bowl 50?'''
question = '''Who is the lead character in the Uncharted series?'''
question = '''Who played Gandalf in the Lord of the Rings movies?'''
question = '''What was Nirvana's last studio album called?''' # TODO test with beamsearch 3. with beam search if I have only 1 allowed token the other beams could generate something illegal and than crash
# TODO understand the crash is due to beam search and abandon those illegal paths. or try to explore always different triples with beam search???


question = '''How many members were there in the original NSYNC music group?'''

question = 'Who influenced Nirvana band?'

question = 'Is Johnny Depp or Brad Pitt older?'
#question = 'Who is the mother of the director of film Polish-Russian War (Film)?' # Małgorzata Braunek
#question = 'Which film came out first, Blind Shaft or The Mask Of Fu Manchu?' # The Mask Of Fu Manchu
question = 'When did John V, Prince Of Anhalt-Zerbst\'s father die?' # 12 June 1516
#question = 'Is Johnny Depp older than Brad Pitt?'

question = '''During which years did Napoleon hold power in France?'''
question = '''When was Barack Obama president?'''

prompted_texts = [model_config.apply_prompt_template(question)]

In [14]:
print(prompted_texts[0])

<|im_start|>system
You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.

Process:

    You receive an input question.

    You get some context about the entities in the questions using description and short description.

    You determine the reasoning path needed to answer the question based on the information available.

    You explicitly list relevant facts, one per line, starting with "Fact:".

    You explain your reasoning step by step and provide a detailed answer, justifying it based on the facts.

    You conclude with a short and concise answer that must be grounded by the facts you found.

You have to answer only basing on the facts you find in the knowledge base.
If you cannot find relevant facts in the knowledge base to support an answer you stop and you reply: "I don't know.".
After generating facts, please reason on top of the facts you find and avoid generating lot of useless facts.

You m

In [10]:
inputs = model_config.tokenizer(prompted_texts, return_tensors='pt', padding=True, padding_side='right')
inputs = inputs.to(model_config.model.device)

In [11]:
model_config.model.device

device(type='cuda', index=0)

In [12]:
torch.cuda.empty_cache()

In [16]:
num_beams = 3

auto_streamer = streamer if num_beams == 1 else None

states = ConstrainedStateList(
    [ConstrainedState(
                begin_pattern = model_config.switch_pattern,
                end_pattern = model_config.newline_token,
                cache_index = DictIndex(end_of_triple=index_config.index.end_of_triple),
                subtree_cache = DictIndex(end_of_triple=index_config.index.end_of_triple),
                oneleaf_cache = DictIndex(end_of_triple=index_config.index.end_of_triple)
            ) for _ in range(num_beams)],
            num_beams=num_beams,
            batch_size = 1,
            pad_token_id = model_config.tokenizer.eos_token_id)

constrained_processor = ConstrainedLogitsProcessor(
    index=index_config.index,
    end_token=model_config.newline_token, states=states, tokenizer=model_config.tokenizer)
logits_processor_list = LogitsProcessorList([
    constrained_processor
])

model_config.model.eval()
start = time.time()

with torch.no_grad():

    pdb.set_trace() # b /opt/conda/envs/development/lib/python3.13/site-packages/transformers/generation/utils.py:3984
    out = model_config.model.generate(
        **inputs,
        logits_processor=logits_processor_list,
        max_new_tokens=800,
        streamer = auto_streamer,
        do_sample = False,
        temperature = None,
        top_k=None,
        num_beams=num_beams,
        #num_beam_groups = num_beams // 2,
        ## TODO group beam search works differently with logitsprocessors: input ids shape is group shape = num_beams / num_beam_groups
        #diversity_penalty=0.5,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        #stopping_criteria=stopping_criteria,
        use_cache=True,
        top_p=None,
        min_p=None,
        #past_key_values=past_key_values,
        kwargs = {'constrained_state': states}, # passing state
    )

print('Elapsed', time.time() - start)

> /tmp/ipykernel_8925/4180010291.py(29)<module>()
     27 with torch.no_grad():
     28 
---> 29     pdb.set_trace() # b /opt/conda/envs/development/lib/python3.13/site-packages/transformers/generation/utils.py:3984
     30     out = model_config.model.generate(
     31         **inputs,



ipdb>  b


Num Type         Disp Enb   Where
1   breakpoint   keep yes   at /opt/conda/envs/development/lib/python3.13/site-packages/transformers/generation/utils.py:3984
	breakpoint already hit 1 time


ipdb>  b /workspace/notebooks/secondment/ctrie.py:687


Breakpoint 2 at /workspace/notebooks/secondment/ctrie.py:687


...

ipdb>  self.beam_idx[0][0,:]


tensor([ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, 

ipdb>  self.beam_idx[0][:,0]


tensor([0, 0, 0], device='cuda:0', dtype=torch.int32)


ipdb>  exit


In [ ]:
# debug for updating to transformers 4.51
'''1   breakpoint   keep yes   at /opt/conda/envs/development/lib/python3.13/site-packages/transformers/generation/utils.py:3984
	breakpoint already hit 1 time

ipdb>  b /workspace/notebooks/secondment/ctrie.py:687

ipdb>  p self.beam_idx

tensor([[[ 0, -1, -1,  ..., -1, -1, -1],
         [ 0, -1, -1,  ..., -1, -1, -1],
         [ 0, -1, -1,  ..., -1, -1, -1]]], device='cuda:0', dtype=torch.int32)

ipdb>  p len(self.beam_idx)

1

ipdb>  p self.num_beams * self.batch_size

3

ipdb>  self.beam_idx.shape

torch.Size([1, 3, 800]) 1 batch_size, 3 num_beams, 800 max_new_tokens # verifica nel codice e empiricamente

# todo capire a che generazione siamo e prendere gli indici così
self.beam_idx[batch_num][:,generation_num]'''

In [14]:
torch.cuda.empty_cache()

In [15]:
_from = len(inputs.input_ids[0]) # 0
for i in range(out.shape[0]):
    print('-'*30, sum(out[i][_from:]), len(out[i][_from:]))
    print(model_config.tokenizer.decode(out[i][_from:]))

------------------------------ tensor(492040, device='cuda:0') 117
Reasoning: To answer this question, I need to find the start and end years of Barack Obama's presidency. Let's start with the description.
Fact: <Barack Obama> <description> <president of the United States from 2009 to 2017> .
Now I have all the information I need to give a grounded answer.
Long answer: Barack Obama served as the President of the United States from 2009 to 2017.

Answer: 2009 to 2017.<|im_end|>
------------------------------ tensor(633566, device='cuda:0') 117
Reasoning: To answer this question, I need to find the start and end years of Barack Obama's presidency. Let's start with the description.
Fact: <Barack Obama> <description> <president of the United States from 2009 to 2017> .
Now I have all the information I need to give a grounded answer.
Long answer: Barack Obama was the president of the United States from 2009 to 2017.

Answer: 2009 to 2017.<|im_end|><|endoftext|>


In [72]:
for i, triple in enumerate(states[0].generated_triples):
    print(i, model_config.tokenizer.decode(triple)[:-1], triple, end='\n')

0  <Barack Obama> <position held> <President of the United States> . [366, 3428, 473, 7105, 29, 366, 3487, 5644, 29, 366, 26137, 315, 279, 3639, 4180, 29, 659, 198]
1  <Barack Obama> <date of birth> <1961-08-04T00:00:00Z> . [366, 3428, 473, 7105, 29, 366, 1028, 315, 7194, 29, 366, 16, 24, 21, 16, 12, 15, 23, 12, 15, 19, 51, 15, 15, 25, 15, 15, 25, 15, 15, 57, 29, 659, 198]
2  <Barack Obama> <significant event> <First inauguration of Barack Obama> . [366, 3428, 473, 7105, 29, 366, 90545, 1538, 29, 366, 5338, 59710, 315, 23539, 7105, 29, 659, 198]
3  <Barack Obama> <significant event> <Second inauguration of Barack Obama> . [366, 3428, 473, 7105, 29, 366, 90545, 1538, 29, 366, 15666, 59710, 315, 23539, 7105, 29, 659, 198]


# Debug unconstrained

In [16]:
num_beams = 1

auto_streamer = streamer if num_beams == 1 else None

model_config.model.eval()
start = time.time()

with torch.no_grad():

    #pdb.set_trace()
    out = model_config.model.generate(
        **inputs,
        #logits_processor=logits_processor_list,
        max_new_tokens=800,
        streamer = auto_streamer,
        do_sample = False,
        temperature = 0.0,
        top_k=None,
        num_beams=num_beams,
        #num_beam_groups = num_beams // 2,
        ## TODO group beam search works differently with logitsprocessors: input ids shape is group shape = num_beams / num_beam_groups
        #diversity_penalty=0.5,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        #stopping_criteria=stopping_criteria,
        use_cache=True,
        #past_key_values=past_key_values,
        #kwargs = {'constrained_state': states}, # passing state
    )

Loading Qwen/Qwen2.5-1.5B-Instruct
<|im_start|>system
You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.

Process:

    You receive an input question.

    You determine the reasoning path needed to answer the question based on the information available.

    You explicitly list relevant facts, one per line, starting with "Fact:".

    You explain your reasoning step by step and provide a detailed answer, justifying it based on the facts.

    You conclude with a short and concise answer.

Handling missing facts:

    If no relevant facts are found in the knowledge base, you:
    a) Provide an answer based on general knowledge, explicitly stating that no supporting facts were found.
    b) If you do not know the answer or are not confident enough, simply reply: "I don't know."

You must always follow these instructions precisely and ensure your responses adhere strictly to this prompt.
<|im_end|>
<|im_sta